In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
!pip install langchain

In [ ]:
import pandas as pd
from langchain import PromptTemplate
from os import path

dataset_csv_filename = "/content/sample_data/friends-1-227.csv"

data = pd.read_csv(dataset_csv_filename)

In [ ]:
prompt_template = PromptTemplate.from_template(
    "<s>[INST] {other_lines} [/INST] {rachel_lines} </s>"
)

In [ ]:
def build_data_item(other_lines, rachel_lines):
    return prompt_template.format(other_lines=other_lines, rachel_lines=rachel_lines)


def clear_unwanted_characters(text):
    return text.replace('"', "")


def preprocess_data_item(text):
    return clear_unwanted_characters(text) + " "

In [ ]:
len(data.index)

In [ ]:
rachel = "Rachel"

global_output = pd.DataFrame(columns=["text"])

global_pointer = -1
other_lines = ""
rachel_lines = ""
total_lines_count = len(data.index)

local_pointer = 0

while global_pointer < total_lines_count:
    global_pointer += local_pointer

    if global_pointer == total_lines_count:
        break

    local_pointer = 0
    other_lines = ""
    rachel_lines = ""

    current_row_index = global_pointer + local_pointer

    if current_row_index >= total_lines_count:
        break

    row = data.iloc[current_row_index]

    while rachel_lines == "" or rachel in row["Speaker"]:
        preprocessed_line = preprocess_data_item(row["Dialogue"])
        if row["Speaker"] == rachel:
            rachel_lines += preprocessed_line
        else:
            other_lines += preprocessed_line

        local_pointer += 1
        current_row_index = global_pointer + local_pointer
        if current_row_index >= total_lines_count:
            break
        row = data.iloc[current_row_index]

    global_output = pd.concat(
        [global_output, pd.DataFrame([build_data_item(other_lines, rachel_lines)])],
        ignore_index=True,
    )

In [ ]:
global_output.to_csv("/content/sample_data/friends-1-227-preprocessed.csv", index=False)

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"

dataset_name = "/content/sample_data/friends-1-227-preprocessed.csv"

# Fine-tuned model name
new_model = "llama-2-7b-rachel"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load dataset
dataset = pd.read_csv("/content/sample_data/friends-1-227-preprocessed.csv")

In [ ]:
# drop text column
dataset = dataset.drop(["text"], axis=1)

In [ ]:
# rename column name
dataset = dataset.rename(columns={"0": "text"})

In [ ]:
dataset

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,3.110400
50,2.277800
75,2.493300
100,1.953500
125,2.387100
150,2.003800
175,2.291200
200,1.952700
225,2.344400
250,1.896700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=2048, training_loss=2.072581110522151, metrics={'train_runtime': 1073.3502, 'train_samples_per_second': 7.631, 'train_steps_per_second': 1.908, 'total_flos': 2.178672447602688e+16, 'train_loss': 2.072581110522151, 'epoch': 1.0})

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
input_texts = [
    "Come on Rach, you don't have what, ten minutes?",
    "Rach, are you okay?",
    "Hey uh, Rach, funny story. I ah, bumped into Joanna on the street yesterday.",
    "Rachel always cries!",
    "We were on a break!",
    "Oh, Rachel, actually, y'know what, forget it.",
    "Do you remember the day we had dessert?",
]

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
from transformers import pipeline, logging
import re

# Initialize the pipeline with your model and tokenizer
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=50, num_return_sequences=1)


for prompt in input_texts:
    # Generate the response
    result = pipe(prompt)
    generated_text = result[0]['generated_text']

    clean_response = generated_text.replace(prompt, "").replace(tokenizer.eos_token, "").strip()

    clean_response = clean_response.split("  ")[0]

    clean_response = clean_response.replace("[/]", "").replace("[//", "").strip()

    clean_response = re.sub(r"\[//.*?\]", "", clean_response).strip()

    print(f"Prompt: {prompt}")
    print(f"Response: {clean_response}\n")


Prompt: Come on Rach, you don't have what, ten minutes?
Response: I'm sorry. Pheebs, I'm sorry.

Prompt: Rach, are you okay?
Response: Yeah, I'm fine. I'm just a little shaken up.

Prompt: Hey uh, Rach, funny story. I ah, bumped into Joanna on the street yesterday.
Response: She said she’s been dating this guy, and he’s really nice.

Prompt: Rachel always cries!
Response: Rachel

Prompt: We were on a break!
Response: It was not a break. It was a break.

Prompt: Oh, Rachel, actually, y'know what, forget it.
Response: I'm not gonna do that. I don't want to be the one to break up your relationship. I mean, I don't think that would be

Prompt: Do you remember the day we had dessert?
Response: Oh, I remember it. Wow, that was a good day.

